In [9]:
def exportSubredditInfo(subreddit):
    import numpy as np
    import pandas as pd

    import requests, json, time, datetime
    import praw
    
    # login credentials
    r_creds = {
        "id" : "",
        "secret" : "",
        "user" : "Adventurous_Party788",
        "pass" : ""
    }
    
    creds_file = open('creds.json', "r")
    r_creds = json.loads(creds_file.read())
    
    reddit = praw.Reddit(
        client_id = r_creds['id'],
        client_secret = r_creds['secret'],
        username = r_creds['user'],
        password = r_creds['pass'],
        user_agent = 'michael'
    )
    
    # access subreddit
    subredditObj = reddit.subreddit(subreddit)
    
    categories = ['hot','new','top','rising']
    dfs = []
    
    # loop through each category
    for category in categories:
        if category == 'hot':
            threads = list(subredditObj.hot(limit=10))
        elif category == 'new':
            threads = list(subredditObj.new(limit=10))
        elif category == 'top':
            threads = list(subredditObj.top(limit=10))
        else:
            threads = list(subredditObj.rising(limit=10))
        titles = [post.title for post in threads]
        num_coms = [post.num_comments for post in threads]
        body_text = [post.selftext for post in threads]
        body_word_count = [len(post.selftext.split()) for post in threads]
        up_votes = [post.ups for post in threads]
        age = [(time.time() - post.created_utc)/60/1440 for post in threads]
        url = [post.url for post in threads]

        category_dict = {
            'title': titles,
            'body_text' : body_text,
            'body_word_count' : body_word_count,
            'num_comments' : num_coms,
            'up_votes' : up_votes,
            'age' : age,
            'url' : url,
            'category' : category,
            'subreddit' : subreddit
        }

        # create and add dataframe to dfs list
        dfs.append(pd.DataFrame(category_dict))
    
    # combine all dataframes
    df = dfs[0].append(dfs[1:])
    
    # add 'all_text' column w/ title, post body, comments
    all_text = []
    for index, row in df.iterrows():
        all_text.append(row['title'] + '\n' + row['body_text'])
    df['all_text'] = all_text
    
    # export data to csv
    df.to_csv('./data/' + subreddit + '_subreddit.csv', encoding='utf-8', index=False)

In [10]:
exportSubredditInfo('remnantgame')

/var/folders/8t/9lr88cgn3xb54w91878m0c1m0000gn/T/ipykernel_36775/1512642044.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfs[0].append(dfs[1:])
